In [5]:
import numpy as np
import pandas as pd
match=pd.read_csv('matches.csv')
delevery=pd.read_csv('deliveries.csv')
total_score=delevery.groupby(['match_id','inning']).sum()['total_runs'].reset_index()
total_score=total_score[total_score['inning']==1]
merged_scorecard=match.merge(total_score[['match_id','total_runs']],left_on='id',right_on='match_id')
total_score['target_runs'] = total_score['total_runs'] + 1

teams=['Royal Challengers Bengaluru','Punjab Kings','Sunrisers Hyderabad','Chennai Super Kings','Kolkata Knight Riders','Mumbai Indians','Rajasthan Royals','Lucknow Super Giants','Delhi Capitals','Gujarat Titans']
match['team1']=match['team1'].str.replace('Delhi Daredevils','Delhi Capitals')
match['team2']=match['team2'].str.replace('Delhi Daredevils','Delhi Capitals')
match['team1']=match['team1'].str.replace('Deccan Chargers','Sunrisers Hyderabad')
match['team2']=match['team2'].str.replace('Deccan Chargers','Sunrisers Hyderabad')
match['team1']=match['team1'].str.replace('Kings XI Punjab','Punjab Kings')
match['team2']=match['team2'].str.replace('Kings XI Punjab','Punjab Kings')
match['team1']=match['team1'].str.replace('Royal Challengers Bangalore','Royal Challengers Bengaluru')
match['team2']=match['team2'].str.replace('Royal Challengers Bangalore','Royal Challengers Bengaluru')
match=match[match['team1'].isin(teams)]
match=match[match['team2'].isin(teams)]

match=match[['id','season','city','team1','team2','winner']]
delevery['current_score'] = 0  
delevery.loc[delevery['inning'] == 2, 'current_score'] = (
    delevery[delevery['inning'] == 2].groupby('match_id')['total_runs'].cumsum()
)
delevery = delevery.merge(total_score[['match_id', 'target_runs']], on='match_id', how='left')
delevery['runs_left'] = delevery['target_runs'] - delevery['current_score']
delevery['runs_left'] = delevery['runs_left'].clip(lower=0)

delevery_ining2=delevery.loc[delevery['inning']==2]
delevery_ining2 = delevery[delevery['inning'] == 2].copy()
delevery_ining2['over'] = delevery_ining2['over'] + 1
delevery_ining2['balls_left']=126-(delevery_ining2['over']*6+delevery_ining2['ball'])
delevery_ining2['wicket_left'] = 10
delevery_ining2['wickets_lost'] = delevery_ining2.groupby(['match_id', 'inning'])['is_wicket'].cumsum()
delevery_ining2['wicket_left'] = 10 - delevery_ining2['wickets_lost']

delevery_ining2 = delevery_ining2.merge(
    match[['id', 'winner']], left_on='match_id', right_on='id', how='left'
)
delevery_ining2.rename(columns={'winner': 'winning_team'}, inplace=True)
delevery_ining2.pop('id')
delevery_ining2.pop('dismissal_kind')
delevery_ining2.pop('fielder')
delevery_ining2.pop('extras_type')
delevery_ining2['CRR']=(delevery_ining2['current_score'])*6/(120-delevery_ining2['balls_left'])
delevery_ining2['RRR']=(delevery_ining2['runs_left'])*6/(delevery_ining2['balls_left'])
delevery_ining2 = delevery_ining2.merge(
    match[['id', 'city']],  
    left_on='match_id',
    right_on='id',
    how='left'
)
delevery_ining2.drop(columns=['id'], inplace=True)
delevery_ining2

,match_id,inning,batting_team,bowling_team,over,ball,batter,bowler,non_striker,batsman_runs,...,current_score,target_runs,runs_left,balls_left,wicket_left,wickets_lost,winning_team,CRR,RRR,city
0,335982,2,Royal Challengers Bangalore,Kolkata Knight Riders,1,1,R Dravid,AB Dinda,W Jaffer,1,...,1,223,222,119,10,0,Kolkata Knight Riders,6.000000,11.193277,Bangalore
1,335982,2,Royal Challengers Bangalore,Kolkata Knight Riders,1,2,W Jaffer,AB Dinda,R Dravid,0,...,2,223,221,118,10,0,Kolkata Knight Riders,6.000000,11.237288,Bangalore
2,335982,2,Royal Challengers Bangalore,Kolkata Knight Riders,1,3,W Jaffer,AB Dinda,R Dravid,0,...,2,223,221,117,10,0,Kolkata Knight Riders,4.000000,11.333333,Bangalore
3,335982,2,Royal Challengers Bangalore,Kolkata Knight Riders,1,4,W Jaffer,AB Dinda,R Dravid,1,...,3,223,220,116,10,0,Kolkata Knight Riders,4.500000,11.379310,Bangalore
4,335982,2,Royal Challengers Bangalore,Kolkata Knight Riders,1,5,R Dravid,AB Dinda,W Jaffer,1,...,4,223,219,115,10,0,Kolkata Knight Riders,4.800000,11.426087,Bangalore
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
125736,1426312,2,Kolkata Knight Riders,Sunrisers Hyderabad,10,5,SS Iyer,AK Markram,VR Iyer,1,...,110,114,4,61,8,2,Kolkata Knight Riders,11.186441,0.393443,Chennai
125737,1426312,2,Kolkata Knight Riders,Sunrisers Hyderabad,10,6,VR Iyer,AK Markram,SS Iyer,1,...,111,114,3,60,8,2,Kolkata Knight Riders,11.100000,0.300000,Chennai
125738,1426312,2,Kolkata Knight Riders,Sunrisers Hyderabad,11,1,VR Iyer,Shahbaz Ahmed,SS Iyer,1,...,112,114,2,59,8,2,Kolkata Knight Riders,11.016393,0.203390,Chennai
125739,1426312,2,Kolkata Knight Riders,Sunrisers Hyderabad,11,2,SS Iyer,Shahbaz Ahmed,VR Iyer,1,...,113,114,1,58,8,2,Kolkata Knight Riders,10.935484,0.103448,Chennai


In [6]:
def result(row):
    return 1 if row['batting_team']==row['winning_team'] else 0


In [7]:
delevery_ining2['result']=delevery_ining2.apply(result,axis=1)
final_data = delevery_ining2[['inning', 'batting_team', 'bowling_team','city', 'runs_left', 'balls_left', 'wicket_left', 'CRR', 'RRR', 'result']]
final_data = final_data.sample(final_data.shape[0])
final_data.sample()

,inning,batting_team,bowling_team,city,runs_left,balls_left,wicket_left,CRR,RRR,result
87316,2,Chennai Super Kings,Kolkata Knight Riders,Abu Dhabi,122,88,9,8.625,8.318182,0


In [10]:
X = final_data.iloc[:,:-1]
y = final_data.iloc[:,-1]
from sklearn.model_selection import train_test_split
X_train,X_test,y_train,y_test = train_test_split(X,y,test_size=0.2,random_state=1)
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import OneHotEncoder
column_transformer = ColumnTransformer([
    ('onehot', OneHotEncoder( drop='first'), ['batting_team', 'bowling_team', 'city'])
]
,remainder='passthrough')
X_train

,inning,batting_team,bowling_team,city,runs_left,balls_left,wicket_left,CRR,RRR
22065,2,Delhi Daredevils,Deccan Chargers,Delhi,136,90,8,6.600000,9.066667
80070,2,Royal Challengers Bangalore,Mumbai Indians,Bengaluru,63,32,7,8.522727,11.812500
118859,2,Punjab Kings,Lucknow Super Giants,Lucknow,46,9,5,8.324324,30.666667
2763,2,Kings XI Punjab,Royal Challengers Bangalore,Bangalore,74,80,9,7.950000,5.550000
102191,2,Sunrisers Hyderabad,Chennai Super Kings,Mumbai,64,44,9,7.184211,8.727273
...,...,...,...,...,...,...,...,...,...
4292,2,Deccan Chargers,Delhi Daredevils,Delhi,158,97,9,9.652174,9.773196
82492,2,Royal Challengers Bangalore,Kings XI Punjab,Chandigarh,136,100,10,11.400000,8.160000
25803,2,Rajasthan Royals,Chennai Super Kings,Jaipur,108,44,6,7.026316,14.727273
104651,2,Chennai Super Kings,Punjab Kings,Mumbai,74,32,6,7.772727,13.875000
